# Find My Next Home - Property Search

This notebook scrapes property listings and visualizes them on an interactive map.

## Import Libraries

In [1]:
from homeharvest import scrape_property
import folium
import pandas as pd
import requests
import time
from datetime import datetime
import os

## Choose Transit Time Method

We'll use free open-source routing services that don't require API keys:
- **OSRM (driving)** - Free, no API key needed
- **OpenRouteService** - Free with optional API key for higher limits
- Falls back to distance estimates if services are unavailable

In [2]:
# No API key needed! We'll use free open-source routing services
print("✓ Using free routing services (OSRM) - no API key required")

✓ Using free routing services (OSRM) - no API key required


## Scrape Properties

Scrape all listings in Westchester, NY.

In [3]:
properties = scrape_property(
    location="Westchester County, NY",
    listing_type="for_sale",  # for_sale, for_rent, pending, sold
    past_days=30  # Get listings from past 30 days
)

properties.to_csv("results.csv", index=False)
print(f"Found {len(properties)} properties")

Found 257 properties


## Preview the Data

In [4]:
# Display first few rows and check column names
print("Available columns:")
print(properties.columns.tolist())
print("\nFirst few rows:")
properties.head()

Available columns:
['property_url', 'property_id', 'listing_id', 'permalink', 'mls', 'mls_id', 'status', 'mls_status', 'text', 'style', 'formatted_address', 'full_street_line', 'street', 'unit', 'city', 'state', 'zip_code', 'beds', 'full_baths', 'half_baths', 'sqft', 'year_built', 'days_on_mls', 'list_price', 'list_price_min', 'list_price_max', 'list_date', 'pending_date', 'sold_price', 'last_sold_date', 'last_sold_price', 'last_status_change_date', 'last_update_date', 'assessed_value', 'estimated_value', 'tax', 'tax_history', 'new_construction', 'lot_sqft', 'price_per_sqft', 'latitude', 'longitude', 'neighborhoods', 'county', 'fips_code', 'stories', 'hoa_fee', 'parking_garage', 'agent_id', 'agent_name', 'agent_email', 'agent_phones', 'agent_mls_set', 'agent_nrds_id', 'broker_id', 'broker_name', 'builder_id', 'builder_name', 'office_id', 'office_mls_set', 'office_name', 'office_email', 'office_phones', 'nearby_schools', 'primary_photo', 'alt_photos']

First few rows:


,property_url,property_id,listing_id,permalink,mls,mls_id,status,mls_status,text,style,...,builder_id,builder_name,office_id,office_mls_set,office_name,office_email,office_phones,nearby_schools,primary_photo,alt_photos
0,https://www.realtor.com/realestateandhomes-det...,3680437841,2989348489,8-Goldwin-St_Rye_NY_10580_M36804-37841,STNY,11617065,FOR_SALE,Active,<NA>,SINGLE_FAMILY,...,<NA>,<NA>,<NA>,O-STNY-10227,SHORT SALE IQ,bscott@myshortsale.com,"[{'number': '(718) 986-0105', 'type': 'Mobile'...",<NA>,https://ap.rdcpix.com/c467d5e7cbaabc18d91f58b4...,https://ap.rdcpix.com/c467d5e7cbaabc18d91f58b4...
1,https://www.realtor.com/realestateandhomes-det...,9022409422,2989627464,6-Aka-10-Rossmore-Ave_Bronxville_NY_10708_M902...,LINY,945529,FOR_SALE,Active,Contractors Special: Your Canvas Awaits in Bro...,MULTI_FAMILY,...,<NA>,<NA>,100050972,O-LINY-REMXPR,Re/Max Prestige Properties,DBerger@remax.net,"[{'number': '9148313090', 'type': 'Office', 'p...",<NA>,https://ap.rdcpix.com/ebd0555240a3f5e27a0fc1fa...,https://ap.rdcpix.com/ebd0555240a3f5e27a0fc1fa...
2,https://www.realtor.com/realestateandhomes-det...,4615639780,2989597617,312A-Spring-St_Ossining_NY_10562_M46156-39780,LINY,896993,FOR_SALE,Active,Enjoy Hudson River views and Sunsets. Being so...,SINGLE_FAMILY,...,<NA>,<NA>,39007,O-LINY-HOULAW05,Houlihan Lawrence BriarCliff Manor,atyourservice@houlihanlawrence.com,"[{'number': '9147627200', 'type': 'Office', 'p...",<NA>,https://ap.rdcpix.com/b1640fc4a71d014672a5420b...,https://ap.rdcpix.com/b1640fc4a71d014672a5420b...
3,https://www.realtor.com/realestateandhomes-det...,9372644804,2989433157,1-Priscella-AKA-31-Primrose-Ave_Yonkers_NY_107...,LINY,943771,FOR_SALE,Active,This is an Estate property being sold strictly...,SINGLE_FAMILY,...,<NA>,<NA>,2878072,O-LINY-HERESE,Weichert Realtors Heritage Pro,<NA>,"[{'number': '9146678996', 'type': 'Office', 'p...",<NA>,https://ap.rdcpix.com/68c350687c4bb254e08a82fb...,https://ap.rdcpix.com/68c350687c4bb254e08a82fb...
4,https://www.realtor.com/realestateandhomes-det...,4141842516,2989258373,1936-Longvue-St_Yorktown-Heights_NY_10598_M414...,LINY,925292,FOR_SALE,Active,Adorable home backing up to the woods yet acce...,SINGLE_FAMILY,...,<NA>,<NA>,39024,O-LINY-HOULAW13,Houlihan Lawrence,zcamaj@houlihanlawrence.com,"[{'number': '9149624900', 'type': 'Office', 'p...",<NA>,https://ap.rdcpix.com/d13743c2f733fba2dd72c830...,https://ap.rdcpix.com/d13743c2f733fba2dd72c830...


## Clean and Prepare Data for Mapping

## Filter by Commute Time to Grand Central

## Pre-compute Station Train Times (Run Once)

This cell computes the fastest train time from each Metro-North station to Grand Central and saves it to a file. **You only need to run this once**, or when the GTFS data updates.

In [5]:
# OPTIONAL: Run this cell once to generate station_train_times.csv
# After running once, you can skip this cell in future runs

import os

# Check if file already exists
if os.path.exists('station_train_times.csv'):
    print("⚠️  station_train_times.csv already exists!")
    print("   Delete it first if you want to regenerate it.")
else:
    print("Computing fastest train times from each station to Grand Central...")
    
    # Load GTFS data
    stops_df = pd.read_csv('gtfsmnr/stops.txt')
    stop_times_df = pd.read_csv('gtfsmnr/stop_times.txt')
    
    # Filter for stations only
    stations = stops_df[stops_df['location_type'] == 0].copy()
    
    # Get morning trains (7-9 AM) arriving at Grand Central
    GRAND_CENTRAL_STOP_ID = '1'
    morning_arrivals = stop_times_df[
        (stop_times_df['stop_id'] == int(GRAND_CENTRAL_STOP_ID)) & 
        (stop_times_df['arrival_time'].str.startswith('0')) &
        (stop_times_df['arrival_time'] >= '07:00:00') & 
        (stop_times_df['arrival_time'] <= '09:00:00')
    ].copy()
    
    trip_ids = morning_arrivals['trip_id'].unique()
    all_stops_for_trips = stop_times_df[stop_times_df['trip_id'].isin(trip_ids)].copy()
    
    # Calculate fastest train time for each station
    station_results = []
    
    for _, station in stations.iterrows():
        station_id = station['stop_id']
        station_name = station['stop_name']
        
        # Find all trains that stop at this station
        station_stops = all_stops_for_trips[all_stops_for_trips['stop_id'] == int(station_id)]
        
        if len(station_stops) == 0:
            continue
        
        fastest_time = float('inf')
        
        for _, stop in station_stops.iterrows():
            trip_id = stop['trip_id']
            departure_time = stop['departure_time']
            
            # Find arrival at Grand Central for this trip
            gc_arrival = all_stops_for_trips[
                (all_stops_for_trips['trip_id'] == trip_id) & 
                (all_stops_for_trips['stop_id'] == int(GRAND_CENTRAL_STOP_ID))
            ]
            
            if len(gc_arrival) > 0:
                dep_parts = departure_time.split(':')
                arr_parts = gc_arrival.iloc[0]['arrival_time'].split(':')
                
                dep_minutes = int(dep_parts[0]) * 60 + int(dep_parts[1])
                arr_minutes = int(arr_parts[0]) * 60 + int(arr_parts[1])
                
                train_duration = arr_minutes - dep_minutes
                if 0 < train_duration < fastest_time:
                    fastest_time = train_duration
        
        if fastest_time != float('inf'):
            station_results.append({
                'stop_id': station_id,
                'stop_name': station_name,
                'stop_lat': station['stop_lat'],
                'stop_lon': station['stop_lon'],
                'fastest_train_minutes': fastest_time
            })
    
    # Save to CSV
    result_df = pd.DataFrame(station_results)
    result_df.to_csv('station_train_times.csv', index=False)
    
    print(f"✓ Saved fastest train times for {len(result_df)} stations to station_train_times.csv")
    print(f"\nSample data:")
    print(result_df.head(10))

⚠️  station_train_times.csv already exists!
   Delete it first if you want to regenerate it.


In [ ]:
# Grand Central Terminal
MAX_COMMUTE_MINUTES = 60
WALKING_BUFFER_MINUTES = 5  # Time to park and walk to platform
MAX_DRIVE_TO_STATION = 20  # Only consider stations within 20 min drive
MIN_DRIVE_TO_STATION = 2  # Minimum reasonable drive time (sanity check)

# Load pre-computed station train times
print("Loading pre-computed station train times...")
if not os.path.exists('station_train_times.csv'):
    print("❌ ERROR: station_train_times.csv not found!")
    print("   Please run the 'Pre-compute Station Train Times' cell first.")
    raise FileNotFoundError("station_train_times.csv not found")

stations = pd.read_csv('station_train_times.csv')
print(f"✓ Loaded {len(stations)} stations with pre-computed train times")

def get_distance_miles(lat1, lon1, lat2, lon2):
    """Fast distance calculation using lat/lon"""
    lat_diff = abs(lat1 - lat2) * 69
    lon_diff = abs(lon1 - lon2) * 54
    return (lat_diff**2 + lon_diff**2)**0.5

def get_driving_time_osrm(origin_lat, origin_lon, dest_lat, dest_lon):
    """Get driving time using OSRM - FREE, no API key needed!"""
    url = f"http://router.project-osrm.org/route/v1/driving/{origin_lon},{origin_lat};{dest_lon},{dest_lat}"
    
    params = {'overview': 'false', 'steps': 'false'}
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        
        if data['code'] == 'Ok' and len(data['routes']) > 0:
            duration_seconds = data['routes'][0]['duration']
            return duration_seconds / 60  # return minutes
        else:
            return None
    except:
        return None

def find_best_station(property_lat, property_lon):
    """
    Find the BEST Metro-North station based on TOTAL commute time (drive + train).
    
    Strategy:
    1. Filter to nearby stations (by distance)
    2. Get actual driving time for top candidates
    3. Choose station with BEST TOTAL COMMUTE (drive + train), not just closest
    """
    # Step 1: Quick filter - calculate straight-line distance to all stations
    distances = []
    for _, station in stations.iterrows():
        distance = get_distance_miles(property_lat, property_lon, 
                                       station['stop_lat'], station['stop_lon'])
        distances.append({
            'station': station,
            'distance': distance,
            'estimated_drive': (distance / 40) * 60  # rough estimate at 40 mph
        })
    
    # Step 2: Sort by distance and take top 5 candidates (increased from 3)
    distances.sort(key=lambda x: x['distance'])
    top_candidates = distances[:5]
    
    # Step 3: Get actual driving time for candidates and calculate TOTAL commute
    best_station = None
    best_total_commute = float('inf')
    best_drive_time = None
    
    for candidate in top_candidates:
        station = candidate['station']
        
        # Skip if no train time data
        if pd.isna(station['fastest_train_minutes']):
            continue
        
        # Only check stations within reasonable driving distance
        if candidate['estimated_drive'] <= MAX_DRIVE_TO_STATION:
            # Get actual driving time from OSRM
            drive_time = get_driving_time_osrm(property_lat, property_lon,
                                                station['stop_lat'], station['stop_lon'])
            
            # SANITY CHECK: If drive time is suspiciously short, use estimate instead
            if drive_time is not None and drive_time < MIN_DRIVE_TO_STATION and candidate['distance'] > 0.5:
                # OSRM returned unrealistic time (e.g., 3 min for 5 miles)
                # This happens when OSRM has bad data or the coordinates are wrong
                drive_time = candidate['estimated_drive']
            elif drive_time is None:
                drive_time = candidate['estimated_drive']
            
            # Calculate TOTAL commute: drive + train + buffer
            total_commute = drive_time + station['fastest_train_minutes'] + WALKING_BUFFER_MINUTES
            
            # Choose station with BEST total commute
            if total_commute < best_total_commute:
                best_total_commute = total_commute
                best_station = station
                best_drive_time = drive_time
            
            # Small delay between API calls
            time.sleep(0.05)
    
    # If no good candidates found, use closest by distance
    if best_station is None:
        best_station = top_candidates[0]['station']
        best_drive_time = top_candidates[0]['estimated_drive']
    
    return best_station, best_drive_time

# Drop rows where latitude or longitude are missing
price_col = 'list_price' if 'list_price' in properties.columns else 'sold_price' if 'sold_price' in properties.columns else 'price'
print(f"\nUsing price column: {price_col}")

properties_clean = properties.dropna(subset=['latitude', 'longitude', price_col])
print(f"Properties with valid coordinates: {len(properties_clean)}")

# Calculate commute times
print(f"\nCalculating commutes to Grand Central via Metro-North...")
print(f"Strategy: Choose station with BEST TOTAL COMMUTE (drive + train)")
print(f"Sanity check: Rejecting OSRM times < {MIN_DRIVE_TO_STATION} min for distances > 0.5 mi")
print(f"Using pre-computed FASTEST train times from station_train_times.csv\n")

commute_data = []

for idx, row in properties_clean.iterrows():
    # Find BEST Metro-North station (optimized for total commute!)
    station, drive_time = find_best_station(row['latitude'], row['longitude'])
    
    if station is not None:
        # INSTANT LOOKUP: Get pre-computed fastest train time
        train_time = station['fastest_train_minutes']
        
        if pd.notna(train_time):
            total_commute = drive_time + train_time + WALKING_BUFFER_MINUTES
            commute_data.append({
                'total_minutes': total_commute,
                'drive_minutes': drive_time,
                'train_minutes': train_time,
                'station_name': station['stop_name'],
                'station_id': station['stop_id']
            })
        else:
            # No train data, estimate
            commute_data.append({
                'total_minutes': drive_time * 2,  # rough estimate
                'drive_minutes': drive_time,
                'train_minutes': None,
                'station_name': station['stop_name'],
                'station_id': station['stop_id']
            })
    else:
        # Fallback
        commute_data.append({
            'total_minutes': 999,
            'drive_minutes': None,
            'train_minutes': None,
            'station_name': 'Unknown',
            'station_id': None
        })
    
    # Progress
    if (idx + 1) % 10 == 0:
        print(f"  Processed {idx + 1}/{len(properties_clean)} properties...")

# Add to dataframe
commute_df = pd.DataFrame(commute_data)
properties_clean['commute_minutes'] = commute_df['total_minutes']
properties_clean['drive_to_station'] = commute_df['drive_minutes']
properties_clean['train_time'] = commute_df['train_minutes']
properties_clean['nearest_station'] = commute_df['station_name']

# Filter by commute time
properties_filtered = properties_clean[properties_clean['commute_minutes'] <= MAX_COMMUTE_MINUTES].copy()

print(f"\n✓ Properties within {MAX_COMMUTE_MINUTES} min commute: {len(properties_filtered)} (from {len(properties_clean)})")
if len(properties_filtered) > 0:
    print(f"\nCommute time stats:")
    print(f"  Min: {properties_filtered['commute_minutes'].min():.0f} minutes")
    print(f"  Max: {properties_filtered['commute_minutes'].max():.0f} minutes")
    print(f"  Average: {properties_filtered['commute_minutes'].mean():.0f} minutes")
    print(f"\nMost common stations:")
    print(properties_filtered['nearest_station'].value_counts().head(5))

## Debug: Test Specific Address

Use this cell to debug a specific property's station selection.

In [ ]:
# OPTIONAL: Debug a specific address
# Find the property in the dataframe and check its station selection

debug_address = "27 Ludlow St"  # Partial address to search for

if 'properties_clean' in locals():
    # Search for the property
    matching = properties_clean[properties_clean['permalink'].str.contains('Ludlow', case=False, na=False)]
    
    if len(matching) > 0:
        print(f"Found {len(matching)} properties matching '{debug_address}':\n")
        
        for idx, prop in matching.iterrows():
            print(f"Address: {prop['permalink']}")
            print(f"Coordinates: {prop['latitude']}, {prop['longitude']}")
            print(f"Selected Station: {prop.get('nearest_station', 'N/A')}")
            print(f"Drive time: {prop.get('drive_to_station', 'N/A'):.1f} min")
            print(f"Train time: {prop.get('train_time', 'N/A'):.1f} min")
            print(f"Total commute: {prop.get('commute_minutes', 'N/A'):.1f} min")
            
            # Now test with top 5 candidates manually
            print(f"\nTesting top 5 nearest stations:")
            prop_lat = prop['latitude']
            prop_lon = prop['longitude']
            
            def get_distance_miles_debug(lat1, lon1, lat2, lon2):
                lat_diff = abs(lat1 - lat2) * 69
                lon_diff = abs(lon1 - lon2) * 54
                return (lat_diff**2 + lon_diff**2)**0.5
            
            distances = []
            for _, station in stations.iterrows():
                distance = get_distance_miles_debug(prop_lat, prop_lon, 
                                                     station['stop_lat'], station['stop_lon'])
                distances.append({
                    'name': station['stop_name'],
                    'distance': distance,
                    'train_time': station['fastest_train_minutes']
                })
            
            distances.sort(key=lambda x: x['distance'])
            for i, candidate in enumerate(distances[:5]):
                estimated_drive = (candidate['distance'] / 40) * 60
                estimated_total = estimated_drive + candidate['train_time'] + 5
                print(f"  {i+1}. {candidate['name']}: {candidate['distance']:.2f} mi away, ~{estimated_drive:.1f} min drive, {candidate['train_time']:.0f} min train, ~{estimated_total:.0f} min total")
            
            print()
    else:
        print(f"No properties found matching '{debug_address}'")
else:
    print("Run the main commute calculation cell first!")

## Visualize Filtered Properties on Map

In [7]:
if not properties_filtered.empty:
    # Get the average latitude and longitude to center the map
    center_lat = properties_filtered['latitude'].mean()
    center_lon = properties_filtered['longitude'].mean()

    # Create a Folium map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10)
    
    # Add Grand Central marker
    folium.Marker(
        location=(40.752998, -73.977056),
        popup="<b>Grand Central Terminal</b>",
        tooltip="Grand Central Terminal",
        icon=folium.Icon(color='red', icon='star')
    ).add_to(m)
    
    # Add Metro-North stations used
    stations_used = properties_filtered['nearest_station'].unique()
    for station_name in stations_used:
        station_info = stations[stations['stop_name'] == station_name]
        if len(station_info) > 0:
            station_row = station_info.iloc[0]
            folium.Marker(
                location=(station_row['stop_lat'], station_row['stop_lon']),
                popup=f"<b>{station_name}</b><br>Metro-North Station",
                tooltip=station_name,
                icon=folium.Icon(color='purple', icon='train', prefix='fa')
            ).add_to(m)

    # Add markers for each property with color coding by commute time
    for idx, row in properties_filtered.iterrows():
        if pd.notna(row['latitude']) and pd.notna(row['longitude']):
            # Clean up permalink for display as address
            address_display = row['permalink'].split('_M')[0].replace('-', ' ').replace('_', ', ').strip()
            
            # Get the appropriate price field
            if 'sold_price' in row and pd.notna(row['sold_price']):
                price = row['sold_price']
                price_label = "Sold Price"
            elif 'list_price' in row and pd.notna(row['list_price']):
                price = row['list_price']
                price_label = "List Price"
            elif 'price' in row and pd.notna(row['price']):
                price = row['price']
                price_label = "Price"
            else:
                price = None
                price_label = "Price"
            
            # Color code by commute time
            commute_time = row['commute_minutes']
            if commute_time <= 45:
                marker_color = 'green'
            elif commute_time <= 60:
                marker_color = 'orange'
            else:
                marker_color = 'blue'
            
            # Build popup text with detailed commute info
            popup_parts = [f"<b>Address:</b> {address_display}"]
            popup_parts.append(f"<b>Total Commute:</b> {commute_time:.0f} min")
            popup_parts.append(f"<b>Nearest Station:</b> {row['nearest_station']}")
            if pd.notna(row['drive_to_station']):
                popup_parts.append(f"  • Drive to station: {row['drive_to_station']:.0f} min")
            if pd.notna(row['train_time']):
                popup_parts.append(f"  • Train time: {row['train_time']:.0f} min")
            popup_parts.append(f"  • Walking buffer: {WALKING_BUFFER_MINUTES} min")
            if price:
                popup_parts.append(f"<b>{price_label}:</b> ${price:,}")
            if pd.notna(row.get('beds')):
                popup_parts.append(f"<b>Beds:</b> {row['beds']}")
            if pd.notna(row.get('full_baths')):
                popup_parts.append(f"<b>Baths:</b> {row['full_baths']}")
            popup_parts.append(f"<a href='{row['property_url']}' target='_blank'>View Listing</a>")
            
            popup_text = "<br>".join(popup_parts)
            
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=popup_text,
                tooltip=f"{address_display} ({commute_time:.0f} min via {row['nearest_station']})",
                icon=folium.Icon(color=marker_color, icon='home', prefix='fa')
            ).add_to(m)

    # Add legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; width: 220px; height: 170px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <p style="margin:0"><b>Commute Time Legend</b></p>
    <p style="margin:5px 0"><i class="fa fa-home" style="color:green"></i> ≤ 45 min total</p>
    <p style="margin:5px 0"><i class="fa fa-home" style="color:orange"></i> 46-60 min total</p>
    <p style="margin:5px 0"><i class="fa fa-train" style="color:purple"></i> Metro-North Station</p>
    <p style="margin:5px 0"><i class="fa fa-star" style="color:red"></i> Grand Central</p>
    <p style="margin:5px 0; font-size:11px; font-style:italic">*Includes drive + train + 5min buffer</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    # Display the map
    display(m)
else:
    print("No properties within commute time to plot.")